The 2021 and 2020 raw data from ghcnd ~100mb each, are too large for github. Can download and extract 2021.csv.gz & 2020.csv.gz from https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/ 

In [1]:
using CSV
using DataFrames
using Statistics
using DataFramesMeta

Load data:

In [4]:
daily_2020 = CSV.File("2020.csv",header=0) |> DataFrame;
daily_2021 = CSV.File("2021.csv",header=0) |> DataFrame;
append!(daily_2021,daily_2021)
countries = CSV.File("fips-10-4-to-iso-country-codes.csv") |> DataFrame;

Select wanted data and rename appropriately, filter for only avg temp & precipitation data.

In [6]:
df = daily_2021;
select!(df,:Column1,:Column2,:Column3,:Column4);
rename!(df,[:station,:date,:element,:value]);
filter!(:element => element -> (element == "TAVG" || element == "PRCP"),df);

Change from 10th of Celsius and tenths of mm -> Celsius and mm

In [7]:
df.value = df.value./10;

Add new column for fips country code, which is a substring of the station names.

In [8]:
df.fips = SubString.(string.(df.station),1,2);

Find mean value for avg temp & precipitation for each country.

In [9]:
climate = combine(groupby(df,[:fips,:element]), :value=>mean=>:value);

Make long data frame into wide data frame, with seperate columns for tavg and prcp.
Doing this by making two subsets and joining them.

In [10]:
tavg = filter(:element => element -> (element == "TAVG"),climate);
rename!(tavg,:value=>:tavg);
select!(tavg,:fips,:tavg);
prcp = filter(:element => element -> (element == "PRCP"),climate);
rename!(prcp,:value=>:prcp);
select!(prcp,:fips,:prcp);

climate_wide = outerjoin(tavg,prcp,on=:fips);

Join the climate and country data together.

In [11]:
data = outerjoin(climate_wide, countries, on = :fips)

,fips,tavg,prcp,iso3166,Name
,SubStri…,Float64?,Float64?,String?,String?
1,AE,30.6095,0.0,AE,United Arab Emirates
2,AF,19.3022,3.32,AF,Afghanistan
3,AG,22.9381,0.783375,DZ,Algeria
4,AJ,17.3267,10.9522,AZ,Azerbaijan
5,AL,17.2345,missing,AL,Albania
6,AM,10.7996,3.05077,AM,Armenia
7,AO,24.737,12.404,AO,Angola
8,AQ,27.5725,9.63018,AS,American Samoa
9,AR,15.6661,6.17386,AR,Argentina


In [13]:
CSV.write("climate.csv",data)

"climate.csv"